In [17]:
import pandas as pd
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

# Step 1: Load the JSON file using Pandas and extract the text data
data = pd.read_json('Testfil_FINAL.json')
description_text = data['description_text']

# Step 2: Preprocess the text data
processed_data = [simple_preprocess(text) for text in description_text]

# Step 3: Train the Word2Vec model
model = Word2Vec(processed_data, min_count=1, vector_size=100)

# Step 4: Save the trained model
model.save('word2vec_model_desc_text')

# You can now use the trained model for various tasks, such as word similarity or keyword-in-context searches.


In [6]:
import nltk
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from nltk.text import Text

# Download NLTK resources
nltk.download('punkt')

# Preprocess the text and tokenize into sentences
#corpus = "Är du intresserad av kundbehov och hur detta kan skapa bättre användarupplevelser?På FLIR får du möjligheten att arbeta med engagerade kollegor där företagskulturen präglas av öppenhet och prestigelöshet.Vi söker dig som vill arbeta inom User Research där du förväntas vara analytisk och utåtriktad.Sök redan idag då urval sker löpande!Rollen innebär att du ska vara med och identifiera samt utvärdera framtidens produkter och vara en evangelist för detta inom produktutvecklingen på FLIR Systems.Du kommer arbeta nära en kollega som jobbat i rollen under en längre period samt samarbeta med personer i andra roller inom industridesign, interaktionsdesign, produktledning, marknadsföring samt andra ingenjörsdiscipliner.Du kommer utföra analytiska undersökningar och ibland innebär det en del resande, både i Sverige och utomlands.Det gäller ungefär 20-25 dagar om året.Den här rekryteringsprocessen hanteras av Academic Work och FLIRs önskemål är att alla samtal och mail kring tjänsten går till Academic Work Uppdraget är en del av vår personaluthyrning.Du kommer vara anställd av Academic Work och arbeta som konsult hos FLIR.FLIRs intention är att du på sikt ska kunna bli anställd direkt hos dem, förutsatt att alla parter är nöjda med samarbetet.Du utför studier för att identifiera kundgrupper men analyserar även färdiga prototyper för att se så att de stämmer in med vad vald målgrupps behov.En stor del av rollen innebär att förstå kundernas behov, kunna uttrycka behoven samt bidra till att de omsätts i praktiken.Du skapar vidare användarscenarios, användarprofiler och liknande.Kunderna är utspridda runt om i världen och därför innebär jobbet en del resor för att kunna träffa samt intervjua dem.Det är din uppgift att dra slutsatser kring kundbehoven.Det kan handla om att svara på frågor såsom, \"Hur ser deras arbetsdag ut?\", \"Vilka problem möter de i arbetet?\", \"Hur löser våra produkter deras problem?\".Du skissar därefter upp en historia kring användaren som på ett målande samt kreativt sätt beskriver kunden.Dessa kundinsikter levererar du bl.a. vidare till designteamet och produktledning.Din roll är också att kontinuerligt utvärdera prototyper, koncept av både mjukvaror och fysiska produkter, för att se om den är rätt utformad för vald målgrupp.Tjänsten passar dig som söker en roll där du får använda dina analytiska och metodiska kunskaper och samtidigt vara utåtriktad och nyfiken.Här finns möjligheten att vara kreativ, ha inflytande samt arbeta med spännande och högteknologiska produkter med kunden i fokus.VI SÖKER DIG SOM * Har en avklarad utbildning inom psykologi, människa-data interaktion (HCI), beteendevetenskap, interaktionsdesign, eller liknande * Har erfarenhet och kunskap av kvalitativa och kvantitativa metoder * Har intresse för teknik och produktutveckling * Har god förmåga att illustrera och kommunicera ett budskap * Har mycket goda kunskaper i engelska då all dokumentation och interaktion kommer ske i detta språk Det är meriterande om du.* Har kommit i kontakt med produktutveckling, interaktionsdesign eller industridesign * Har erfarenhet av att arbeta med A/B testning, google analytics eller liknande analysverktyg Vi lägger stor vikt vid dina personliga egenskaper för denna tjänst.Som person har du en empatisk förmåga att sätta dig in i andra människors behov kombinerat med en hög känska för kvalite.Du ser dig själv som en analytisk person som kan bidra med inspiration och har lätt för att skapa nya kontaktytor.Du behöver ha förmågan att tänka kritiskt och inte vara rädd för att föreslå förslag till förändringar.Vidare är det viktigt att du är social samt framåt då du kommer behöva ta kontakt med olika människor i olika roller både inom och utanför FLIR.I din roll ingår även att presentera material för kollegor vilket gör att du behöver vara duktig kommunikativt, både i tal och skrift.ÖVRIG INFORMATION * Start: Januari * Omfattning: 6 månader med goda chanser till förlängning * Placering: Täby * Sök tjänsten genom att klicka på Ansök nedan.Vi går igenom urvalet löpande och annonsen kan stängas ner innan tjänsten är tillsatt om vi gått över till urvals och intervjufasen.Skriv annonstiteln i rubriken och kopiera gärna in länken till annonsen i ditt mail.För generella funderingar kring din ansökan/vår rekryteringsprocess finns vi ofta tillgängliga i chatten här på webben under vardagar, hör gärna av dig där!Academic Work är Home of the Young Professionals.Vi vänder oss till dig som är akademiker och fortfarande studerar på högskola/universitet, är nyutexaminerad eller har några års arbetslivserfarenhet.* Vårt erbjudande till dig som heltidskonsult hos Academic Work inkluderar utöver kollektivavtalsenliga villkor och dedikerad konsultchef också rabatter och erbjudanden genom vår konsultportal.Läs mer om vårt erbjudande här SKYDD FÖR DIN PERSONLIGA INTEGRITET När du skickar in din ansökan till Academic Work godkänner och samtycker du till att Academic Work behandlar dina personuppgifter enligt Personuppgiftslagen.Läs mer under vår PuL-sida."
corpus = 'analytisk'
sentences = [word_tokenize(sentence) for sentence in Text(corpus)]

# Train Word2Vec model
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1)

# Get word embeddings
target_word = "analytisk"

word_embedding = model.wv[target_word]

# Perform KWIC search
context_words = model.wv.similar_by_vector(word_embedding, topn=5)
context_words = [word for word, _ in context_words]

kwic_results = []
for sentence in sentences:
    if target_word in sentence:
        idx = sentence.index(target_word)
        context = " ".join(sentence[max(0, idx - 5):idx] + sentence[idx + 1:min(idx + 6, len(sentence))])
        kwic_results.append((context, sentence))

# Print KWIC results
for context, sentence in kwic_results:
    print(context, "[", target_word, "]", " ".join(sentence))


[nltk_data] Downloading package punkt to /Users/home/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


KeyError: "Key 'analytisk' not present"

In [27]:
import pandas as pd
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from nltk.tokenize import sent_tokenize

# Step 1: Load the JSON file using Pandas and extract the text data
data = pd.read_json('Testfil_FINAL.json')
description_text = data['description_text']

# Step 2: Tokenize sentences in the text data
sentences = []
for text in description_text:
    text_sentences = sent_tokenize(text)
    sentences.extend(text_sentences)

# Step 3: Preprocess the text data
processed_data = [simple_preprocess(sentence) for sentence in sentences]

# Step 4: Train the Word2Vec model
model = Word2Vec(processed_data, min_count=1, vector_size=100)

# Step 5: Save the trained model
model.save('word2vec_model_desc_sentence')

# You can now use the trained model for various tasks, such as word similarity or keyword-in-context searches.


In [20]:
with open('sentences.txt', 'w', encoding='utf-8') as file:
    for sentence in sentences:
        file.write(sentence + '\n')

In [24]:
# Load the saved Word2Vec model
model = Word2Vec.load('word2vec_model_desc_sentence')

# Define a target sentence for similarity comparison
target_sentence = "analytisk"

# Preprocess the target sentence
processed_target = simple_preprocess(target_sentence)

# Find similar sentences
similar_sentences = model.wv.most_similar(positive=processed_target, topn=5)

# Print the similar sentences
for sentence, similarity in similar_sentences:
    print(f"Similarity: {similarity:.2f} Sentence: {sentence}")


Similarity: 0.93 Sentence: pedagogisk
Similarity: 0.91 Sentence: noggrann
Similarity: 0.90 Sentence: strukturerad
Similarity: 0.90 Sentence: kommunikativ
Similarity: 0.88 Sentence: serviceinriktad


In [31]:
# Load the saved Word2Vec model
model = Word2Vec.load('word2vec_model_desc_sentence')

# Define a target sentence for similarity comparison
target_sentence = "Du är diplomatisk, har god analytisk förmåga och kan enkelt röra dig mellan detaljer och helheter."

# Preprocess the target sentence
processed_target = simple_preprocess(target_sentence)

print(processed_target)

# Find similar sentences
similar_sentences = []
for sentence in processed_data:
    similarity = model.wv.n_similarity(processed_target, sentence)
    similar_sentences.append((sentence, similarity))

# Sort the similar sentences based on similarity score
similar_sentences = sorted(similar_sentences, key=lambda x: x[1], reverse=True)

# Print the top 5 similar sentences
for sentence, similarity in similar_sentences[:5]:
    original_sentence = ' '.join(sentence)
    print(f"Similarity: {similarity:.2f} Sentence: {original_sentence}")


['du', 'är', 'diplomatisk', 'har', 'god', 'analytisk', 'förmåga', 'och', 'kan', 'enkelt', 'röra', 'dig', 'mellan', 'detaljer', 'och', 'helheter']


ZeroDivisionError: At least one of the passed list is empty.